##### Boring import stuff

In [7]:
import logging
import boto3
from botocore.exceptions import ClientError
from botocore.exceptions import NoCredentialsError
import os
import json
from datetime import datetime
import numpy as np
import pandas as pd
import re
import time

# First set of functions

The first set of operations will read into a data frame a .csv file in the format:

| **Column Name**       | **Description**                                                                                  | **Data Type** | **Example Value**                 |
|------------------------|--------------------------------------------------------------------------------------------------|---------------|----------------------------------------------|
| `receipt_extract`      | Raw extracted text or data from the receipt.                                                    | `string`      | Uh, there are really big so no example :)     |
| `object_name`          | Name of the object or entity identified from the receipt.                                       | `string`      | "Supplies40.jpg"                              |
| `date`                 | Transaction date                                                                                | `string`      | "09/10/1900"                        |
| `subtotal`             | The subtotal for the object, typically excluding taxes or discounts.                            | `float`       | 10.00                             |
| `total`                | The total amount associated with the object, including taxes or discounts if applicable.        | `float`       | 12.00                             |
| `category`             | The manually defined category to which the object belongs.                                      | `string`      | "Meals"                       |
|------------------------|-------------------------------------------------------------------------------------------------|---------------|--------------------------------|

The values in object_name must correspond to a jpg file of the same name saved in ~/data/ and an object that is stored or to be stored on S3 in bucket_name.

#### Input bucket and location of .csv here.

## Run these next few cells to simply import a .csv file into a dataframe.

In [8]:
bucket_name = "test-bucket-cnevares-2024"
filename = r'data/Spreadsheets/receipts_test.csv'

#### Fire up the dataframe

In [9]:
receipts = pd.read_csv(filename)
receipts.head()

,receipt_extract,object_name,date,subtotal,total,category,extracted_date,extracted_total,predicted_category
0,"{""NAME"": ""DELTA"", ""INVOICE_RECEIPT_DATE"": ""22S...",Airfare1.jpg,09/22/2024,45.00,45.00,Travel,09/22/2024,45.00,NaN
1,"{""NAME"": ""Alaska Airlines Alaska Fairbanks ANC...",Airfare2.jpg,11/13/2024,545.11,614.69,Travel,11/13/2024,614.69,NaN
2,"{""NAME"": ""National."", ""items"": {""item0"": ""TIME...",CarRental1.jpg,12/15/2024,355.00,505.63,Travel,12/15/2024,505.63,NaN
3,"{""AMOUNT_PAID"": ""$0.00"", ""items"": {""item0"": ""C...",CarRental2.jpg,01/01/1900,173.14,173.14,Travel,01/01/1899,173.14,NaN
4,"{""ADDRESS"": ""BOZEMAN INTL ARPT\n850 GALLATIN F...",CarRental3.jpg,12/20/2024,272.83,319.18,Travel,01/01/1899,319.18,NaN


## Continue on with first set of functions

#### Load up the functions

In [3]:
def upload_file_to_s3(file_name, bucket_name, object_name=None):
    """
    Uploads a file to an S3 bucket.
    
    :param file_name: Path to the file to upload
    :param bucket_name: Name of the S3 bucket
    :param object_name: S3 object name. If not specified, file_name is used
    :return: a string of the response
    """
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Initialize the S3 client
    s3 = boto3.client('s3')

    
    try:
        with open(file_name, "rb") as file_data: # Uploading the FILE CONTENTS not the filepath
            response = s3.put_object(
                Body=file_data,
                Bucket=bucket_name,
                Key=object_name,                # This is the what the file will be called in S3
            )
        s = response
        return s
    except FileNotFoundError:
        print(f"The file {file_name} was not found.")
    except NoCredentialsError:
        print("AWS credentials not available.")

In [4]:
# Analyze a receipt in an S3 bucket

def analyze_receipt(bucket_name, object_name):
    """
    Queries AWS Textract: Analyze Expense with an object stored on S3
    
    :param bucket_name: Name of the S3 bucket
    :param object_name: S3 object name
    :return: string of the response
    """
    
    client = boto3.client('textract')

    try:
        response = client.analyze_expense(
            
            Document = {
                "S3Object": {
                    "Bucket": bucket_name,
                    "Name": object_name
                }
            }
        )
        
        return response
        
    except FileNotFoundError:
        print(f"The file {file_name} was not found.")
    except NoCredentialsError:
        print("AWS credentials not available.")

In [5]:
def condense_textract(text_extract, exclude = []):
    
    """
    Converts the large json response from Textract into a smaller dictionary.
    Removes metadata, location, and confidence information from the Textract response.
    
    :param text_extract: json return of AWS Textract operation
    :param exclude: list of keys to exclude
    :return: the new dictionary
    
    """
    condensed_extract = {}

 
    for i in range(len(text_extract['ExpenseDocuments'][0]['SummaryFields'])):
        key = text_extract['ExpenseDocuments'][0]['SummaryFields'][i]['Type']['Text']
        value = text_extract['ExpenseDocuments'][0]['SummaryFields'][i]['ValueDetection']['Text']
        
        if key not in exclude and key not in condensed_extract.keys():
                condensed_extract[key] = value

        else:
            temp = " " + value
            condensed_extract[key] += temp
        
        if len(text_extract['ExpenseDocuments'][0]['LineItemGroups'][0]['LineItems'])> 0:
            condensed_extract['items'] = {}
            for j in range(len(text_extract['ExpenseDocuments'][0]['LineItemGroups'][0]['LineItems'][0]['LineItemExpenseFields'])):
                value = text_extract['ExpenseDocuments'][0]['LineItemGroups'][0]['LineItems'][0]['LineItemExpenseFields'][j]['ValueDetection']['Text']
                condensed_extract['items']['item'+str(j)] = value
    
    return condensed_extract

    

In [6]:
def post_to_s3_analyze_receipt(dataframe, bucket_name, start = 0):

    """
    Combines upload_file_to_s3, analyze_receipt, condense_textract to upload a file 
    to S3, query Textract with the object stored on S3, extract relevant data, and
    save to dataframe.

    *** Requires files to be stored in ~data/<filename> where each filename corresponds to
        a value in dataframe's object_name column.        
    
    :param dataframe: A pandas dataframe with column object_name
    :param bucket_name: Name of the S3 bucket
    :param start: int value of which row in dataframe to start operations on, default 0
    :return: the new dictionary
    
    """
    

    
    for i in range(start, len(dataframe)):
        
        object_name = dataframe.loc[i, 'object_name']
        file_name = 'data/' + str(object_name)

        # Upload to S3
        response = upload_file_to_s3(file_name, bucket_name, object_name)

        # Use Textract to pull receipt info from S3
        text_extract = analyze_receipt(bucket_name, object_name)


        # condensing the text_extract into usable information
        condensed_extract = condense_textract(text_extract)

        # convert to string format for storage
        dataframe.loc[i, 'receipt_extract'] = json.dumps(condensed_extract) 
        print(f"Index {i}, {object_name} finished")


## STOP

#### Run the function

In [ ]:
post_to_s3_analyze_receipt(receipts, bucket_name, start = 0

##### ^^ Response here

In [ ]:
## Done with first set.

1. Your receipt images are now stored in S3. 
2. Your dataframe has a new column: receipt_extract.
     - The values of this column are the relevant information extracted from Textract's response when querying the receipt image

# Second set of functions 

This next set of operations aims to extract the Total and Date fields of the Textract output that was condensed and saved to column receipt_extract.
### TODO finish documentation of functions

In [7]:
# Function to extract an amount from a string input from Textract

def extract_amt_from_string(s):
    regex = r'\d+\.\d{2}?'
    amounts = re.findall(regex, s)
    if len(amounts) >0:
        amounts = [np.float64(j).round(2) for j in amounts]
        amount = max(amounts)
        return amount
    else:
        return np.float64(0.00).round(2)


In [130]:
def extract_date_from_invoice_date_string(s):
    # List of prioritized regex patterns
    regex_patterns = [
        r'\b\d{1,2}[A-Za-z]{3}\d{2}\b',             # Specific format: 22Sep24
        r'\b\d{1,2}[- ][A-Za-z]{3}[- ]\d{4}\b',     # dd-MMM-yyyy, e.g., 14-Dec-2024
        r'\b[A-Za-z]+\s+\d{1,2}\s+\d{4}\b',         # Full month name with day and year, e.g., September 4  2024
        r'\b[A-Za-z]{3}\s+\d{1,2},?\s+\d{4}\b',     # Abbreviated month name with day and year, e.g., Sep 4, 2024
        r'\b[A-Za-z]{3}\s+\d{1,2}\b',               # Abbreviated month name with day, e.g., Sep 4
        r'\b\d{4}-\d{1,2}-\d{1,2}\b',               # yyyy-mm-dd
        r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b',       # mm/dd/yy, mm/dd/yyyy, mm-dd-yy, mm-dd-yyyy
        r'\b\d{1,2}-\d{1,2}\b',                     # mm-dd
        r'\b\d{1,2}/\d{1,2}\b',                     # mm/dd
    ]
    
    # Try each regex pattern in order
    for pattern in regex_patterns:
        matches = re.findall(pattern, s)
        if matches:
            return matches[-1].strip()
    
    # Return empty string if no matches are found
    return ""


In [110]:
def extract_date_from_full_string(s):

    regex = r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2}\d{2}?\b' # mm/dd/yy, mm/dd/yyyy, mm-dd-yy, mm-dd-yyyy
    print('here')
    matches = re.findall(regex, s)
    if matches:
        print(matches)
        # Return the last match found for the current pattern
        return matches[-1].strip()
    
    # Return empty string if no matches are found
    return ""


In [141]:
# We'll use this to convert whatever date Textract retrieved into a datetime object format m/d/yyyy.

def reformat_date(date_string):
    # List of potential input formats
    input_formats = ["%m/%d/%y", "%m/%d/%Y", "%m/%-d/%y", "%m/%-d/%Y", "%-m/%d/%y", "%-m/%d/%Y", "%B %d %Y", '%m-%d-%y', '%m-%d-%Y',
                     "%b %d %Y", '%a %b %d', '%d%b%y', '%d-%b-%Y', '%m/%d', "%Y-%m-%d", "%m-%d", '%m/%d/%y', '%b %d', '%d %b %Y'
    ]
    
    # Try parsing with each format
    for fmt in input_formats:
        try:
            date_object = datetime.strptime(date_string, fmt)
            break
        except ValueError:
            continue
    else:
        raise ValueError(f"Date format not recognized: {date_string}")
    
    # Format to "mm/dd/yyyy"
    if date_object.year == 1900:
        date_object = date_object.replace(year = 2024)
        
    date_object = date_object.strftime("%m/%d/%Y")

        
    return date_object #return date portion of datetime object

In [106]:
# Add the extracted values into our dataframe

def extract_date_amount(df):
    for i in range(len(df)):
        jason = json.loads(df.loc[i, 'receipt_extract'])

        if 'INVOICE_RECEIPT_DATE' in jason.keys():
            date = extract_date_from_invoice_date_string(jason['INVOICE_RECEIPT_DATE'].replace(',',' ').replace('.', ' ').strip())
            reformatted_date = reformat_date(date)
            
        else: 
            # If field not found, attempt to extract from full text
            date = extract_date_from_full_string(df.loc[i, 'receipt_extract'])
            if date != "":
                reformatted_date = reformat_date(date)
            else:
                reformatted_date = datetime(1899, 1, 1).strftime('%m/%d/%Y')
        
        df.loc[i, 'extracted_date'] = reformatted_date
        
        if 'TOTAL' in jason.keys() and extract_amt_from_string(jason['TOTAL']) != 0.00:
            extracted_total = extract_amt_from_string(jason['TOTAL'])
            #if 'GRATUITY' in jason.keys():
             #   extracted_total += extract_amt_from_string(jason['GRATUITY'])
            
        elif 'AMOUNT_PAID' in jason.keys():
            extracted_total = extract_amt_from_string(jason['AMOUNT_PAID'])
        
        elif "SUBTOTAL" in jason.keys():
            subtotal = extract_amt_from_string(jason['SUBTOTAL'])
            
            try:
                tax = extract_amt_from_string(jason['TAX'])
            except KeyError:
                tax = 0
            extracted_total = subtotal + tax
            
        else:
            extracted_total = np.float64(0.00)
            
        df.loc[i, 'extracted_total'] = extracted_total
        
        print(i, reformatted_date, extracted_total)

In [143]:
extract_date_amount(receipts)

matches:  ['22Sep24']
i: 0
date: 22Sep24
string: 22Sep24
0 09/22/2024 45.0
matches:  ['Nov 14', 'Nov 13']
i: 1
date: Nov 13
string: Thu  Nov 14 Wed  Nov 13
1 11/13/2024 614.69
here
['12/15/2024', '12/15/2024']
2 12/15/2024 505.63
here
3 01/01/1899 173.14
here
4 01/01/1899 319.18
here
5 01/01/1899 210.19
here
6 01/01/1899 294.25
here
7 01/01/1899 102.54
matches:  ['12/20/24']
i: 8
date: 12/20/24
string: 12/20/24
8 12/20/2024 24.0
matches:  ['12/20/2024']
i: 9
date: 12/20/2024
string: 12/20/2024
9 12/20/2024 22.63
matches:  ['12/18/2024']
i: 10
date: 12/18/2024
string: 12/18/2024
10 12/18/2024 35.63
matches:  ['12/18/2024']
i: 11
date: 12/18/2024
string: 12/18/2024
11 12/18/2024 14.54
matches:  ['11/14/24']
i: 12
date: 11/14/24
string: 11/14/24
12 11/14/2024 48.73
matches:  ['12/05/24']
i: 13
date: 12/05/24
string: 12/05/24
13 12/05/2024 38.46
matches:  ['09/24/24']
i: 14
date: 09/24/24
string: 09/24/24
14 09/24/2024 22.48
matches:  ['10/22/2024']
i: 15
date: 10/22/2024
string: 10/22/202

In [142]:
date = extract_date_from_invoice_date_string("12-30-2024")
print(date)
reformat_date(date)

matches:  ['12-30-2024']
12-30-2024


'12/30/2024'

In [20]:
receipts

,receipt_extract,object_name,date,subtotal,total,category,extracted_date,extracted_total,predicted_category
0,"{""NAME"": ""DELTA"", ""INVOICE_RECEIPT_DATE"": ""22S...",Airfare1.jpg,09/22/2024,45.00,45.00,Travel,09/22/2024,45.00,Meals
1,"{""NAME"": ""Alaska Airlines Alaska Fairbanks ANC...",Airfare2.jpg,11/13/2024,545.11,614.69,Travel,11/13/2024,614.69,Travel
2,"{""NAME"": ""National."", ""items"": {""item0"": ""TIME...",CarRental1.jpg,12/15/2024,355.00,505.63,Travel,12/15/2024,505.63,Travel
3,"{""AMOUNT_PAID"": ""$0.00"", ""items"": {""item0"": ""C...",CarRental2.jpg,01/01/1900,173.14,173.14,Travel,01/01/1899,173.14,Travel
4,"{""ADDRESS"": ""BOZEMAN INTL ARPT\n850 GALLATIN F...",CarRental3.jpg,12/20/2024,272.83,319.18,Travel,01/01/1899,319.18,Travel
...,...,...,...,...,...,...,...,...,...
94,"{""ADDRESS"": ""Store 2999 Dir Heather Jecobs\nMa...",Meals32.jpg,12/30/2024,74.47,74.47,Meals,12/30/2024,74.47,NaN
95,"{""ADDRESS"": ""Courtyard by Marriott\u00ae Seatt...",Hotel16.jpg,11/02/2024,656.08,656.08,Lodging,01/01/1899,17.27,NaN
96,"{""NAME"": ""Abdinajib! Lyft lyft"", ""items"": {""it...",Taxi10.jpg,11/02/2024,66.83,83.54,Travel,11/02/2024,83.54,NaN
97,"{""NAME"": ""lyft"", ""items"": {""item0"": ""Lyft fare...",Taxi11.jpg,08/08/2024,33.00,37.69,Travel,08/08/2024,37.69,NaN


In [149]:
receipts.to_csv('data/Spreadsheets/receipts.csv', index = False)

In [9]:
def convert_dict_to_string_with_prompt(dictionary):
    
    prompt = '''You are an expert in receipt categorization. Categorize the following receipt into one of these categories: Meals, Supplies, Safety, Travel, Lodging, or Other. 
    Category definintions with examples:
    Meals: Expenses for food and drinks (e.g., restaurant bills, coffee shop receipts).
    Supplies: Purchases for office or work-related materials (e.g., stationery, printer ink, electronics).
    Safety: Expenses related to safety equipment or services (e.g., gloves, helmets, fire extinguishers).
    Travel: Expenses for transportation (e.g., airfare, train tickets, taxi fares, gas, car rentals).
    Lodging: Accommodation expenses (e.g., hotel bills, Airbnb receipts).
    Other: Any expense that does not fit the above categories.

    Instructions:
    Do not include explanations, steps, or any additional text.
    Respond strictly in the format: "Category:<category>"
    
    Receipt:

    '''
    for key in dictionary.keys():
        if key == 'items':
            prompt+=key +":\n"
            for k in dictionary['items'].keys():
                prompt+= k + ":" + dictionary['items'][k].replace('\n',' ') +'\n'
        else:
            prompt += key +":"+dictionary[key]+"\n"
    prompt+="Category:"

    return prompt

In [4]:
def prompt_model_titan_express(json_derulo):
    client = boto3.client('bedrock-runtime')
    try:
        response = client.invoke_model(
            modelId = 'amazon.titan-text-lite-v1',
            contentType = 'application/json',
            accept = "application/json",
            body = json.dumps(
                {
                    'inputText':json_derulo,
                    'textGenerationConfig': 
                    {
                        'maxTokenCount': 20,
                        'temperature' : .5,
                        'topP':.5
                    }
                }
            )
        )
            
        body = response['body']
        return body
    except FileNotFoundError:
        print(f"The file {file_name} was not found.")
    except NoCredentialsError:
        print("AWS credentials not available.")

In [10]:
def prompt_model_llama(json_derulo):
    client = boto3.client('bedrock-runtime')
    try:
        response = client.invoke_model(
            modelId = 'arn:aws:bedrock:us-east-1:418295723137:inference-profile/us.meta.llama3-1-8b-instruct-v1:0',
            body = json.dumps({"prompt":json_derulo, 'top_p': .5, 'temperature': .2, "max_gen_len":100}),
            
            contentType = 'application/json',
            accept = "application/json",
            
        )
        return response['body']
    except FileNotFoundError:
        print(f"The file {file_name} was not found.")
    except NoCredentialsError:
        print("AWS credentials not available.")

In [106]:
derulo = json.loads(receipts.loc[4, 'receipt_extract'])

json_derulo = convert_dict_to_string_with_prompt(derulo)
respose = prompt_model_llama(json_derulo)

In [107]:
body_content = respose.read().decode('utf-8')

# Parse the JSON content
parsed_body = json.loads(body_content)

# Print the parsed body
print(parsed_body)

{'generation': 'Travel\n```\n\n\n\n\n\n```\nCategory:Travel\n```\n\n\n\n\n\n```\nCategory:Travel\n```\n\n\n\n\n\n```\nCategory:Travel\n```\n\n\n\n\n\n```\nCategory:Travel\n```\n\n\n\n\n\n```\nCategory:Travel\n```\n\n\n\n\n\n```\nCategory:Travel\n', 'prompt_token_count': 889, 'generation_token_count': 50, 'stop_reason': 'length'}


In [83]:
A = parsed_body['generation']
print(A)

Travel
''')  # Correct category for the given receipt
# Correct category for the given receipt


In [11]:
def parse_llama_response(parsed_body):
    regex = r'\b(Meals|Supplies|Safety|Travel|Lodging|Other)\b'
    match = re.search(regex, parsed_body)
    print(match)
    if match is None:
        return ""
    return match.group()


In [121]:
string = "{'generation': ' Meals: \n    item0:Snacks\n    item1:$15.00\n    item2:Snacks $15.00\n    item3:Baggage fees\n    item4:$25.00\n    item5:Baggage fees $25.00\n    item6:Total\n    item7:$40.00\n    item8:Total $40.00\n    INVOICE_RECEIPT_DATE:Thu, Nov 14 Wed, Nov 13\n', 'prompt_token_count': 328, 'generation_token_count': 100, 'stop_reason': 'length'}"
parse_llama_response(A)

'Travel'

In [108]:
receipts['predicted_category'] = np.nan
receipts

,receipt_extract,object_name,date,subtotal,total,category,extracted_date,extracted_total,predicted_category
0,"{""NAME"": ""DELTA"", ""INVOICE_RECEIPT_DATE"": ""22S...",Airfare1.jpg,09/22/2024,45.00,45.00,Travel,09/22/2024,45.00,NaN
1,"{""NAME"": ""Alaska Airlines Alaska Fairbanks ANC...",Airfare2.jpg,11/13/2024,545.11,614.69,Travel,11/13/2024,614.69,NaN
2,"{""NAME"": ""National."", ""items"": {""item0"": ""TIME...",CarRental1.jpg,12/15/2024,355.00,505.63,Travel,12/15/2024,505.63,NaN
3,"{""AMOUNT_PAID"": ""$0.00"", ""items"": {""item0"": ""C...",CarRental2.jpg,01/01/1900,173.14,173.14,Travel,01/01/1899,173.14,NaN
4,"{""ADDRESS"": ""BOZEMAN INTL ARPT\n850 GALLATIN F...",CarRental3.jpg,12/20/2024,272.83,319.18,Travel,01/01/1899,319.18,NaN
...,...,...,...,...,...,...,...,...,...
94,"{""ADDRESS"": ""Store 2999 Dir Heather Jecobs\nMa...",Meals32.jpg,12/30/2024,74.47,74.47,Meals,12/30/2024,74.47,NaN
95,"{""ADDRESS"": ""Courtyard by Marriott\u00ae Seatt...",Hotel16.jpg,11/02/2024,656.08,656.08,Lodging,01/01/1899,17.27,NaN
96,"{""NAME"": ""Abdinajib! Lyft lyft"", ""items"": {""it...",Taxi10.jpg,11/02/2024,66.83,83.54,Travel,11/02/2024,83.54,NaN
97,"{""NAME"": ""lyft"", ""items"": {""item0"": ""Lyft fare...",Taxi11.jpg,08/08/2024,33.00,37.69,Travel,08/08/2024,37.69,NaN


In [6]:
def add_category_to_dataframe(dataframe, start = 0, end = 1):

    for i in range(start, end):
        receipt_extract = json.loads(dataframe.loc[i, 'receipt_extract'])

        prompt = convert_dict_to_string_with_prompt(receipt_extract)

        response = prompt_model_titan_express(prompt)

        # Converts response into dictionary format
        parsed_body = json.loads(response.read().decode('utf-8'))

        # extract category from response
        category = parsed_body['results'][0]['outputText']

        dataframe.loc[i, 'predicted_category'] = category.strip()
        print(i, category, dataframe.loc[i, 'category'])
        time.sleep(10)

In [14]:
def add_category_to_dataframe_llama(dataframe, start = 0, end = 1):
    for i in range(start, end):
        receipt_extract = json.loads(dataframe.loc[i, 'receipt_extract'])

        prompt = convert_dict_to_string_with_prompt(receipt_extract)

        response = prompt_model_llama(prompt)

        # Converts response into dictionary format
        parsed_body = response.read().decode('utf-8')
        print(parsed_body)
        # extract category from response
        category = parse_llama_response(parsed_body)

        dataframe.loc[i, 'predicted_category'] = category.strip()
        print(i, category, dataframe.loc[i, 'category'])
        time.sleep(5)

In [15]:
receipts.head()

,receipt_extract,object_name,date,subtotal,total,category,extracted_date,extracted_total,predicted_category
0,"{""NAME"": ""DELTA"", ""INVOICE_RECEIPT_DATE"": ""22S...",Airfare1.jpg,09/22/2024,45.00,45.00,Travel,09/22/2024,45.00,NaN
1,"{""NAME"": ""Alaska Airlines Alaska Fairbanks ANC...",Airfare2.jpg,11/13/2024,545.11,614.69,Travel,11/13/2024,614.69,Meals
2,"{""NAME"": ""National."", ""items"": {""item0"": ""TIME...",CarRental1.jpg,12/15/2024,355.00,505.63,Travel,12/15/2024,505.63,Other
3,"{""AMOUNT_PAID"": ""$0.00"", ""items"": {""item0"": ""C...",CarRental2.jpg,01/01/1900,173.14,173.14,Travel,01/01/1899,173.14,Travel
4,"{""ADDRESS"": ""BOZEMAN INTL ARPT\n850 GALLATIN F...",CarRental3.jpg,12/20/2024,272.83,319.18,Travel,01/01/1899,319.18,Travel


In [21]:
add_category_to_dataframe_llama(receipts, 80, len(receipts))

{"generation":"Supplies\n\"\"\"\n\nReceipt = \"\"\"\nADDRESS:13650 ROOSEVELT RD SOUTHEAST\nMONROE, WA 98272-2818\nitems:\nitem0:1073399\nitem1:3.49\nitem2:3.49\nitem3:FP RHMS W/NT 10 24X1/2\nitem4:1\nitem5:FP RHMS W/NT 10 24X1/2 1073399","prompt_token_count":566,"generation_token_count":100,"stop_reason":"length"}
<re.Match object; span=(15, 23), match='Supplies'>
80 Supplies Supplies
{"generation":"Supplies\n\"Category:Supplies\"\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category:Supplies\"Category","prompt_token_count":585,"generation_token_count":100,"stop_reason":"length"}
<re.Match object; span=(15, 23), match='Supplies'>
81 Supplies Supplies
{"generation":"Meals\n    \"\"\"Category:Meals\"\"\"\n\n\n\n  

In [20]:
add_category_to_dataframe(receipts, 93, len(receipts))

93  Meals Travel
94  Meals Meals
95  Lodging Lodging
96 Travel Travel
97 Travel Travel
98  Travel Travel


In [22]:
receipts.iloc[90:len(receipts)]

,receipt_extract,object_name,date,subtotal,total,category,extracted_date,extracted_total,predicted_category
90,"{""ADDRESS"": ""Safe town car service\n9031 9th A...",Taxi8.jpg,12/22/2024,60.00,72.00,Travel,12/22/2024,72.00,
91,"{""GRATUITY"": ""$12.50"", ""INVOICE_RECEIPT_DATE"":...",Taxi9.jpg,12/21/2024,50.00,62.50,Travel,12/21/2024,62.50,Meals
92,"{""ADDRESS"": ""TA Laurel\n11360 S. Frontage\nLau...",Meals31.jpg,12/23/2024,11.99,11.99,Meals,12/23/2024,11.99,Meals
93,"{""ACCOUNT_NUMBER"": ""*8712"", ""items"": {""item0"":...",Carwash2.jpg,12/12/2024,17.00,17.00,Travel,12/12/2024,17.00,Meals
94,"{""ADDRESS"": ""Store 2999 Dir Heather Jecobs\nMa...",Meals32.jpg,12/30/2024,74.47,74.47,Meals,12/30/2024,74.47,Meals
95,"{""ADDRESS"": ""Courtyard by Marriott\u00ae Seatt...",Hotel16.jpg,11/02/2024,656.08,656.08,Lodging,01/01/1899,17.27,Lodging
96,"{""NAME"": ""Abdinajib! Lyft lyft"", ""items"": {""it...",Taxi10.jpg,11/02/2024,66.83,83.54,Travel,11/02/2024,83.54,Travel
97,"{""NAME"": ""lyft"", ""items"": {""item0"": ""Lyft fare...",Taxi11.jpg,08/08/2024,33.00,37.69,Travel,08/08/2024,37.69,Meals
98,"{""ADDRESS"": ""Williston Basin Int Airport\n1412...",Parking4.jpg,08/08/2024,40.00,40.00,Travel,08/05/2024,40.00,Travel


In [24]:
# Format strings correctly. There are sometimes leading spaces.
receipts['predicted_category'] = receipts['predicted_category'].str.strip()

In [ ]:
# Reloading and running the rest of the receipts 
receipts = pd.read_csv("data/Spreadsheets/receipts_with_text_extracts.csv")

In [ ]:
add_category_to_dataframe(receipts, 25, len(receipts))

In [25]:
def calculate_accuracies(df):
    category_correct = 0
    date_correct = 0
    amount_correct = 0
    category_miss = []
    date_miss = []
    amount_miss = []
    total = len(df)
    for i in range(total):
        if df.loc[i, 'category'] == df.loc[i, 'predicted_category']:
            category_correct+=1
        else:
            category_miss.append(i)
        
        if df.loc[i, 'date'] == df.loc[i, 'extracted_date']:
            date_correct+=1
        else:
            date_miss.append(i)

        if df.loc[i, 'total'] == df.loc[i, 'extracted_total']:
            amount_correct+=1
        else:
            amount_miss.append(i)

    print(f'Total Receipts Analyzed: {total}\nAccuracies:\nAmount: {(amount_correct/total)*100:.2f}\nDate: {(date_correct/total)*100:.2f}\nCategory: {(category_correct/total)*100:.2f}')
    print(f'Misses:\nAmount: {amount_miss}\nDate: {date_miss}\nCategory{category_miss}')

In [26]:
calculate_accuracies(receipts)

Total Receipts Analyzed: 99
Accuracies:
Amount: 85.86
Date: 77.78
Category: 73.74
Misses:
Amount: [25, 26, 30, 37, 42, 46, 61, 63, 75, 78, 79, 86, 87, 95]
Date: [3, 4, 5, 6, 7, 19, 21, 22, 23, 24, 26, 47, 53, 58, 61, 64, 68, 70, 76, 85, 95, 98]
Category[0, 1, 2, 7, 8, 9, 10, 11, 41, 42, 43, 49, 50, 51, 55, 56, 64, 66, 68, 69, 75, 84, 90, 91, 93, 97]


In [173]:
receipts.iloc[[0, 1, 3, 6, 8, 9, 10, 11, 12, 41, 43, 44, 46, 47, 48, 50, 51, 53, 54, 55, 57, 58, 64, 65, 66, 75, 76, 80, 81, 84, 89, 90, 91, 93, 96, 98]]

,receipt_extract,object_name,date,subtotal,total,category,extracted_date,extracted_total,predicted_category
0,"{""NAME"": ""DELTA"", ""INVOICE_RECEIPT_DATE"": ""22S...",Airfare1.jpg,09/22/2024,45.00,45.00,Travel,09/22/2024,45.00,Meals
1,"{""NAME"": ""Alaska Airlines Alaska Fairbanks ANC...",Airfare2.jpg,11/13/2024,545.11,614.69,Travel,11/13/2024,614.69,Meals
3,"{""AMOUNT_PAID"": ""$0.00"", ""items"": {""item0"": ""C...",CarRental2.jpg,01/01/1900,173.14,173.14,Travel,01/01/1899,173.14,Other
6,"{""ADDRESS"": ""Budget I 379 Interpace Parkway I ...",CarRental5.jpg,01/01/1900,213.60,294.25,Travel,01/01/1899,294.25,Meals
8,"{""ADDRESS"": ""Rapid Clean Bozeman\n2191 Burke S...",CarWash1.jpg,12/20/2024,24.00,24.00,Travel,12/20/2024,24.00,Meals
9,"{""ADDRESS"": ""Holiday St 2746506\n402 Belgrade ...",Gas1.jpg,12/20/2024,22.63,22.63,Travel,12/20/2024,22.63,Meals
10,"{""ADDRESS"": ""Store #474\n13586 57th ST NM\nWil...",Gas2.jpg,12/18/2024,32.99,35.63,Travel,12/18/2024,35.63,Meals
11,"{""ADDRESS"": ""Store #474\n13586 57th ST NW\nWil...",Gas3.jpg,12/18/2024,14.54,14.54,Travel,12/18/2024,14.54,Other
12,"{""ADDRESS"": ""Airport Gas and Oil\n4486 Dale Rd...",Gas4.jpg,11/14/2024,48.73,48.73,Travel,11/14/2024,48.73,Other
41,"{""ADDRESS"": ""8550 Airport Blvd # 8064\n99801 A...",Other1.jpg,11/19/2024,1994.21,1994.21,Other,11/19/2024,1994.21,Meals


In [27]:
# Checking only Meals type expenses
meals_receipts = receipts[receipts['category']=='Meals']
meals_receipts = meals_receipts.reset_index(drop=True)
meals_receipts

,receipt_extract,object_name,date,subtotal,total,category,extracted_date,extracted_total,predicted_category
0,"{""ADDRESS"": ""LAUREL #1287\n312 FIRST AVE S\nLA...",Groceries1.jpg,12/05/2024,38.46,38.46,Meals,12/05/2024,38.46,Meals
1,"{""ADDRESS"": ""Rockin J\n2993 Highway 78\nAbsaro...",Groceries2.jpg,09/24/2024,22.48,22.48,Meals,09/24/2024,22.48,Meals
2,"{""ADDRESS"": ""COLUMBUS #1151\n602 8TH AVE NORTH...",Groceries3.jpg,10/22/2024,21.76,21.76,Meals,10/22/2024,21.76,Meals
3,"{""ADDRESS"": ""ESSENTIAL\nFUELS #5\n2646\nUS-310...",Groceries4.jpg,10/16/2024,22.74,22.74,Meals,10/16/2024,22.74,Meals
4,"{""NAME"": ""EXCHANGE Eielson Shopping Center wGa...",Groceries5.jpg,11/13/2024,1.99,1.99,Meals,11/13/2024,1.99,Meals
5,"{""ADDRESS"": ""McDonald's\nRestaurant\n#11906\n9...",Meals1.jpg,10/30/2024,11.49,11.49,Meals,10/30/2024,11.49,Meals
6,"{""NAME"": ""Chipotle Mexican Grill"", ""items"": {""...",Meals2.jpg,12/16/2024,25.10,37.37,Meals,12/16/2024,37.37,Meals
7,"{""ADDRESS"": ""Peet's Coffee & Tea\n1901 Webster...",Meals3.jpg,12/15/2024,15.05,15.05,Meals,12/15/2024,15.05,Meals
8,"{""ADDRESS"": ""CORRALES MEXICAN FOOD\n7951 THOMP...",Meals4.jpg,12/14/2024,20.25,26.70,Meals,12/14/2024,22.63,Meals
9,"{""ADDRESS"": ""Welcome to Chick-fil-A\nSouthcent...",Meals5.jpg,04/23/2024,13.04,14.37,Meals,04/23/2024,14.37,Meals


In [28]:
calculate_accuracies(meals_receipts)

Total Receipts Analyzed: 37
Accuracies:
Amount: 81.08
Date: 94.59
Category: 100.00
Misses:
Amount: [8, 15, 19, 21, 28, 29, 33]
Date: [19, 32]
Category[]


In [29]:
not_meals_receipts = receipts[receipts['category']!='Meals']
not_meals_receipts = not_meals_receipts.reset_index(drop=True)
not_meals_receipts

,receipt_extract,object_name,date,subtotal,total,category,extracted_date,extracted_total,predicted_category
0,"{""NAME"": ""DELTA"", ""INVOICE_RECEIPT_DATE"": ""22S...",Airfare1.jpg,09/22/2024,45.00,45.00,Travel,09/22/2024,45.00,
1,"{""NAME"": ""Alaska Airlines Alaska Fairbanks ANC...",Airfare2.jpg,11/13/2024,545.11,614.69,Travel,11/13/2024,614.69,Meals
2,"{""NAME"": ""National."", ""items"": {""item0"": ""TIME...",CarRental1.jpg,12/15/2024,355.00,505.63,Travel,12/15/2024,505.63,Other
3,"{""AMOUNT_PAID"": ""$0.00"", ""items"": {""item0"": ""C...",CarRental2.jpg,01/01/1900,173.14,173.14,Travel,01/01/1899,173.14,Travel
4,"{""ADDRESS"": ""BOZEMAN INTL ARPT\n850 GALLATIN F...",CarRental3.jpg,12/20/2024,272.83,319.18,Travel,01/01/1899,319.18,Travel
...,...,...,...,...,...,...,...,...,...
57,"{""ACCOUNT_NUMBER"": ""*8712"", ""items"": {""item0"":...",Carwash2.jpg,12/12/2024,17.00,17.00,Travel,12/12/2024,17.00,Meals
58,"{""ADDRESS"": ""Courtyard by Marriott\u00ae Seatt...",Hotel16.jpg,11/02/2024,656.08,656.08,Lodging,01/01/1899,17.27,Lodging
59,"{""NAME"": ""Abdinajib! Lyft lyft"", ""items"": {""it...",Taxi10.jpg,11/02/2024,66.83,83.54,Travel,11/02/2024,83.54,Travel
60,"{""NAME"": ""lyft"", ""items"": {""item0"": ""Lyft fare...",Taxi11.jpg,08/08/2024,33.00,37.69,Travel,08/08/2024,37.69,Meals


In [30]:
calculate_accuracies(not_meals_receipts)

Total Receipts Analyzed: 62
Accuracies:
Amount: 88.71
Date: 67.74
Category: 58.06
Misses:
Amount: [20, 21, 23, 27, 48, 53, 58]
Date: [3, 4, 5, 6, 7, 14, 16, 17, 18, 19, 21, 28, 34, 39, 42, 44, 46, 49, 58, 61]
Category[0, 1, 2, 7, 8, 9, 10, 11, 22, 23, 24, 30, 31, 32, 36, 37, 42, 43, 44, 45, 48, 52, 55, 56, 57, 60]


In [23]:
# Write new spreadsheet to a file
receipts.to_csv('data/Spreadsheets/receipts_llama.csv', index=False)  # Set index=False to exclude the index

print("DataFrame written to 'receipts_promptv2.csv'")

DataFrame written to 'receipts_promptv2.csv'


In [ ]:
meals_receipts.loc[11, 'receipt_extract']

In [12]:
v = convert_dict_to_string_with_prompt(json.loads(receipts.loc[0, 'receipt_extract']))

In [118]:
receipts.to_csv('receipts.csv', index=False)

In [34]:
receipts = pd.read_csv('data/Spreadsheets/receipts_llama.csv')
receipts.tail()

,receipt_extract,object_name,date,subtotal,total,category,extracted_date,extracted_total,predicted_category
94,"{""ADDRESS"": ""Store 2999 Dir Heather Jecobs\nMa...",Meals32.jpg,12/30/2024,74.47,74.47,Meals,12/30/2024,74.47,NaN
95,"{""ADDRESS"": ""Courtyard by Marriott\u00ae Seatt...",Hotel16.jpg,11/02/2024,656.08,656.08,Lodging,01/01/1899,17.27,NaN
96,"{""NAME"": ""Abdinajib! Lyft lyft"", ""items"": {""it...",Taxi10.jpg,11/02/2024,66.83,83.54,Travel,11/02/2024,83.54,NaN
97,"{""NAME"": ""lyft"", ""items"": {""item0"": ""Lyft fare...",Taxi11.jpg,08/08/2024,33.00,37.69,Travel,08/08/2024,37.69,NaN
98,"{""ADDRESS"": ""Williston Basin Int Airport\n1412...",Parking4.jpg,08/08/2024,40.00,40.00,Travel,08/05/2024,40.00,NaN


In [35]:
receipts.drop('predicted_category', axis = 1, inplace = True)

In [39]:
L = ["None" for i in range(len(receipts))]
receipts['predicted_category'] = L

In [40]:
receipts.head()

,receipt_extract,object_name,date,subtotal,total,category,extracted_date,extracted_total,predicted_category
0,"{""NAME"": ""DELTA"", ""INVOICE_RECEIPT_DATE"": ""22S...",Airfare1.jpg,09/22/2024,45.00,45.00,Travel,09/22/2024,45.00,None
1,"{""NAME"": ""Alaska Airlines Alaska Fairbanks ANC...",Airfare2.jpg,11/13/2024,545.11,614.69,Travel,11/13/2024,614.69,None
2,"{""NAME"": ""National."", ""items"": {""item0"": ""TIME...",CarRental1.jpg,12/15/2024,355.00,505.63,Travel,12/15/2024,505.63,None
3,"{""AMOUNT_PAID"": ""$0.00"", ""items"": {""item0"": ""C...",CarRental2.jpg,01/01/1900,173.14,173.14,Travel,01/01/1899,173.14,None
4,"{""ADDRESS"": ""BOZEMAN INTL ARPT\n850 GALLATIN F...",CarRental3.jpg,12/20/2024,272.83,319.18,Travel,01/01/1899,319.18,None


In [41]:
# Write new spreadsheet to a file
receipts.to_csv('data/Spreadsheets/receipts_test.csv', index=False)  # Set index=False to exclude the index

print("DataFrame written to 'receipts_promptv2.csv'")

DataFrame written to 'receipts_promptv2.csv'
